In [12]:
import torch
import torch.nn as nn
import torchsummary

import wandb
import numpy as np

# torchvision
from torchvision import datasets, transforms
from torchvision.datasets import ImageFolder
from torch.utils.data.sampler import SubsetRandomSampler
from torch.utils.data import Dataset, DataLoader, random_split


# Device configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [2]:
wandb.login()

wandb: Currently logged in as: dongju_coredottoday (coredottoday-dongju). Use `wandb login --relogin` to force relogin


True

In [3]:
# imagefloder 생성
train_imgfolder = ImageFolder(root='data/train',
                      transform=transforms.Compose([
                          transforms.ToTensor(),
                          transforms.Resize(224),
                          transforms.RandomHorizontalFlip(p=0.8),
#                           transforms.GaussianBlur(kernel_size=(19, 19), sigma=(1.0, 2.0)),
                          transforms.RandomRotation(degrees=(-30, 30), interpolation=transforms.InterpolationMode.BILINEAR, fill=0),
                          transforms.CenterCrop(224)
                      ]))


validation_imgfolder = ImageFolder(root='data/validation',
                          transform=transforms.Compose([
                          transforms.ToTensor(),
                          transforms.Resize(224),
                          transforms.CenterCrop(224)
                      ]))


test_imgfolder = ImageFolder(root='data/test',
                      transform=transforms.Compose([
                          transforms.ToTensor(),
                          transforms.Resize(224),
                          transforms.CenterCrop(224)
                      ]))

In [4]:
# get data_loader
train_data_loader = DataLoader(dataset=train_imgfolder, 
                         batch_size=16, 
                         num_workers=0,
                         shuffle=True,
                         drop_last=True
                        )

valid_data_loader = DataLoader(dataset=validation_imgfolder, 
                         batch_size=16, 
                         num_workers=0
                        )

test_data_loader = DataLoader(dataset=test_imgfolder, 
                         batch_size=1, 
                         num_workers=0
                        )

In [13]:
class VGG16(nn.Module):
    def __init__(self, num_classes=4):
        super(VGG16, self).__init__()
        self.layer1 = nn.Sequential(
            nn.Conv2d(3, 64, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU())
        self.layer2 = nn.Sequential(
            nn.Conv2d(64, 64, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU(), 
            nn.MaxPool2d(kernel_size = 2, stride = 2))
        self.layer3 = nn.Sequential(
            nn.Conv2d(64, 128, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(128),
            nn.ReLU())
        self.layer4 = nn.Sequential(
            nn.Conv2d(128, 128, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(128),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size = 2, stride = 2))
        self.layer5 = nn.Sequential(
            nn.Conv2d(128, 256, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(256),
            nn.ReLU())
        self.layer6 = nn.Sequential(
            nn.Conv2d(256, 256, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(256),
            nn.ReLU())
        self.layer7 = nn.Sequential(
            nn.Conv2d(256, 256, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(256),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size = 2, stride = 2))
        self.layer8 = nn.Sequential(
            nn.Conv2d(256, 512, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(512),
            nn.ReLU())
        self.layer9 = nn.Sequential(
            nn.Conv2d(512, 512, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(512),
            nn.ReLU())
        self.layer10 = nn.Sequential(
            nn.Conv2d(512, 512, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(512),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size = 2, stride = 2))
        self.layer11 = nn.Sequential(
            nn.Conv2d(512, 512, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(512),
            nn.ReLU())
        self.layer12 = nn.Sequential(
            nn.Conv2d(512, 512, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(512),
            nn.ReLU())
        self.layer13 = nn.Sequential(
            nn.Conv2d(512, 512, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(512),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size = 2, stride = 2))
        self.fc = nn.Sequential(
            # nn.Dropout(0.5),
            nn.Linear(7*7*512, 4096),
            nn.ReLU())
        self.fc1 = nn.Sequential(
            # nn.Dropout(0.5),
            nn.Linear(4096, 1024),
            nn.ReLU())
        self.fc2= nn.Sequential(
            nn.Linear(1024, num_classes))
        
    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = self.layer3(out)
        out = self.layer4(out)
        out = self.layer5(out)
        out = self.layer6(out)
        out = self.layer7(out)
        out = self.layer8(out)
        out = self.layer9(out)
        out = self.layer10(out)
        out = self.layer11(out)
        out = self.layer12(out)
        out = self.layer13(out)
        out = out.reshape(out.size(0), -1)
        out = self.fc(out)
        out = self.fc1(out)
        out = self.fc2(out)
        return out

In [14]:
def normal_init(m):
    if isinstance(m, nn.Conv2d):
        torch.nn.init.normal_(m.weight, mean=0, std=0.01)
        if m.bias is not None:
            torch.nn.init.constant_(m.bias, 0)
    elif isinstance(m, nn.Linear):
        torch.nn.init.normal_(m.weight, mean=0, std=0.01)
        if m.bias is not None:
            torch.nn.init.constant_(m.bias, 0)
    elif isinstance(m, nn.BatchNorm2d):
        torch.nn.init.constant_(m.weight, 1)
        torch.nn.init.constant_(m.bias, 0)

In [15]:
num_classes = 4
num_epochs = 100
batch_size = 16
learning_rate = 0.0001

model = VGG16(num_classes).to(device)
model.apply(normal_init)

# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate, weight_decay = 0.0001)  


# Train the model
total_step = len(train_data_loader)

In [17]:
torchsummary.summary(model, (3, 224, 224))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 64, 224, 224]           1,792
       BatchNorm2d-2         [-1, 64, 224, 224]             128
              ReLU-3         [-1, 64, 224, 224]               0
            Conv2d-4         [-1, 64, 224, 224]          36,928
       BatchNorm2d-5         [-1, 64, 224, 224]             128
              ReLU-6         [-1, 64, 224, 224]               0
         MaxPool2d-7         [-1, 64, 112, 112]               0
            Conv2d-8        [-1, 128, 112, 112]          73,856
       BatchNorm2d-9        [-1, 128, 112, 112]             256
             ReLU-10        [-1, 128, 112, 112]               0
           Conv2d-11        [-1, 128, 112, 112]         147,584
      BatchNorm2d-12        [-1, 128, 112, 112]             256
             ReLU-13        [-1, 128, 112, 112]               0
        MaxPool2d-14          [-1, 128,

In [ ]:
run = wandb.init(
    project="pbl",
    config={
        "learning_rate": learning_rate,
        "eprchs": num_epochs
    }
)

In [9]:
train_datanum = len(train_data_loader)*batch_size
valid_datanum = len(valid_data_loader)*batch_size

In [10]:
# 학습 진행

for epoch in range(num_epochs):
    loss_sum = 0
    correct = 0
    model.train()
    for train_x, train_y in train_data_loader:
        optimizer.zero_grad()
        output = model(train_x.cuda())
        
        loss = criterion(output.cpu(), train_y)
        
        loss.backward()
        optimizer.step()

        loss_sum += loss.item()
        


        predicted = torch.max(output, 1)[1]        
        correct += (train_y == predicted.cpu()).sum()

    print(f'--------------------------------------------------------')
    print(f'Epoch [{epoch+1}/{num_epochs}] Loss: {loss_sum/len(train_data_loader):.4f} Accuracy: {correct / train_datanum:.4f}')
    train_loss = loss_sum/len(train_data_loader)
    train_acc = correct / train_datanum
    
    val_loss_sum = 0
    val_correct = 0
    model.eval()
    
    with torch.no_grad():

        for valid_x, valid_y in valid_data_loader:
            valid_x = valid_x.to('cuda')

            val_output = model(valid_x)
            val_loss = criterion(val_output.cpu(), valid_y)
            val_loss_sum += val_loss.item()
            
            val_predicted = torch.max(val_output, 1)[1]
            val_correct += (valid_y == val_predicted.cpu()).sum()
    

    print(f'Epoch [{epoch+1}/{num_epochs}] val_Loss: {val_loss_sum/len(valid_data_loader):.4f} val_Accuracy: {val_correct / valid_datanum:.4f}')

    validation_loss = val_loss_sum/len(valid_data_loader)
    validation_acc = val_correct / valid_datanum
    
    
    wandb.log({"train_acc":train_acc,"train_loss":train_loss,"validation_acc":validation_acc,"validation_loss":validation_loss})
        
    print('-----------------next-----------------')

/opt/conda/lib/python3.8/site-packages/torchvision/transforms/functional.py:1603: UserWarning: The default value of the antialias parameter of all the resizing transforms (Resize(), RandomResizedCrop(), etc.) will change from None to True in v0.17, in order to be consistent across the PIL and Tensor backends. To suppress this warning, directly pass antialias=True (recommended, future default), antialias=None (current default, which means False for Tensors and True for PIL), or antialias=False (only works on Tensors - PIL will still use antialiasing). This also applies if you are using the inference transforms from the models weights: update the call to weights.transforms(antialias=True).
  warnings.warn(


--------------------------------------------------------
Epoch [1/100] Loss: 0.8198 Accuracy: 0.6954
Epoch [1/100] val_Loss: 0.6257 val_Accuracy: 0.7812
-----------------next-----------------
--------------------------------------------------------
Epoch [2/100] Loss: 0.5265 Accuracy: 0.8111
Epoch [2/100] val_Loss: 0.4991 val_Accuracy: 0.8906
-----------------next-----------------
--------------------------------------------------------
Epoch [3/100] Loss: 0.4254 Accuracy: 0.8470
Epoch [3/100] val_Loss: 1.9831 val_Accuracy: 0.6406
-----------------next-----------------
--------------------------------------------------------
Epoch [4/100] Loss: 0.4406 Accuracy: 0.8484
Epoch [4/100] val_Loss: 0.3328 val_Accuracy: 0.9219
-----------------next-----------------
--------------------------------------------------------
Epoch [5/100] Loss: 0.3707 Accuracy: 0.8764
Epoch [5/100] val_Loss: 0.4465 val_Accuracy: 0.8438
-----------------next-----------------
----------------------------------------

Epoch [43/100] val_Loss: 0.5567 val_Accuracy: 0.9062
-----------------next-----------------
--------------------------------------------------------
Epoch [44/100] Loss: 0.0961 Accuracy: 0.9655
Epoch [44/100] val_Loss: 0.1634 val_Accuracy: 0.9375
-----------------next-----------------
--------------------------------------------------------
Epoch [45/100] Loss: 0.1062 Accuracy: 0.9591
Epoch [45/100] val_Loss: 0.2077 val_Accuracy: 0.9219
-----------------next-----------------
--------------------------------------------------------
Epoch [46/100] Loss: 0.1005 Accuracy: 0.9691
Epoch [46/100] val_Loss: 0.3107 val_Accuracy: 0.9219
-----------------next-----------------
--------------------------------------------------------
Epoch [47/100] Loss: 0.0861 Accuracy: 0.9677
Epoch [47/100] val_Loss: 0.3315 val_Accuracy: 0.9062
-----------------next-----------------
--------------------------------------------------------
Epoch [48/100] Loss: 0.0936 Accuracy: 0.9662
Epoch [48/100] val_Loss: 0.473

--------------------------------------------------------
Epoch [86/100] Loss: 0.0607 Accuracy: 0.9792
Epoch [86/100] val_Loss: 0.3435 val_Accuracy: 0.9219
-----------------next-----------------
--------------------------------------------------------
Epoch [87/100] Loss: 0.0607 Accuracy: 0.9784
Epoch [87/100] val_Loss: 0.1838 val_Accuracy: 0.9062
-----------------next-----------------
--------------------------------------------------------
Epoch [88/100] Loss: 0.0381 Accuracy: 0.9864
Epoch [88/100] val_Loss: 0.2318 val_Accuracy: 0.9375
-----------------next-----------------
--------------------------------------------------------
Epoch [89/100] Loss: 0.0549 Accuracy: 0.9806
Epoch [89/100] val_Loss: 0.2702 val_Accuracy: 0.8750
-----------------next-----------------
--------------------------------------------------------
Epoch [90/100] Loss: 0.0676 Accuracy: 0.9741
Epoch [90/100] val_Loss: 0.1477 val_Accuracy: 0.9531
-----------------next-----------------
------------------------------

In [20]:
# 모델 저장
torch.save(model.state_dict(), 'VGG_custom.pth')

In [22]:
# 모델 불러오기
test_model = VGG16(num_classes)
test_model.load_state_dict(torch.load('VGG_custom.pth'))

<All keys matched successfully>

In [24]:
# 초기화
test_loss = 0
test_correct = 0
test_loss_sum = 0

with torch.no_grad():
    
    for test_x, test_y in test_data_loader:
        test_x = test_x.to('cuda')
        test_y = test_y.to('cuda')

        test_output = test_model.to('cuda')(test_x)
        test_loss = criterion(test_output, test_y)
        test_loss_sum += test_loss.item()
        predicted = torch.max(test_output, 1)[1]
        test_correct += (test_y == predicted).sum()

In [25]:
acc = test_correct / len(test_data_loader)
print(f'Test Accuracy: {acc.item()}')

Test Accuracy: 0.21875
